In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json




In [176]:
f = open('selected_team_match_info.jsonlines', 'r')
alldata = f.readlines()
f.close()

In [ ]:
#json.loads(alldata[0])['picks_bans']

In [177]:
def parseMatch(match, params_list):
    values = [None for _ in range(len(params_list))]
    return_dict = dict(zip(params_list, values))
    for param in params_list:
        try:
            if param == 'radiant_win':
                if match[param]:
                    return_dict[param] = 1
                else:
                    return_dict[param] = 0
            else:
                return_dict[param] = match[param]
        except:
            return_dict[param] = None
                
    return return_dict
        

#print(parseMatch(json.loads(alldata[0]), ['radiant_win', 'huinner']))

In [178]:
def parsePicsBans(match):
    radiant_pick = []
    dire_pick = []
    
    try:
        picks_bans = match['picks_bans']
        for hero in picks_bans:
            if hero['is_pick'] == True:
                if hero['team'] == 1:
                    radiant_pick.append(int(hero['hero_id']))
                else:
                    dire_pick.append(int(hero['hero_id']))
    except:
        pass
    
    return radiant_pick, dire_pick

In [179]:
def generateDataFrame(alldata, params_list):
    params_list = ['match_id'] + params_list
    return_data = pd.DataFrame(columns = params_list)
    
    for match_string in alldata:
        match = json.loads(match_string)
        match_data = parseMatch(match, params_list)
        
        return_data.loc[len(return_data)] = match_data
    
    return return_data

In [ ]:
teams_data = generateDataFrame(alldata, ['radiant_name', 'dire_name', 'radiant_win'])
teams_data.head()

In [ ]:
teams = np.concatenate((teams_data.dire_name.unique(), teams_data.radiant_name.unique())) 
raiant_counts = teams_data.radiant_name.value_counts()
dire_counts = teams_data.dire_name.value_counts()

In [5]:
#READ DATA IF YOU NEED
hdr = pd.read_csv('dotadatadruje.csv')
winrates = pd.read_csv('TimeWinRates.csv')

In [ ]:
only_heroes_data = pd.DataFrame(columns = ['match_id'] + ['hero_' + str(i) for i in range(1,114)] + ['radiant_name', 'dire_name', 'winner'])
winner_cell = 116

error_count = 0

for match_string in alldata:
    match = json.loads(match_string)
    match_results = [0 for i in range(116)] + ['']
    match_results_inverse = [0 for i in range(116)] + ['']
    
    try:
        match_info =  parseMatch(match, ['match_id', 'radiant_win', 'radiant_name', 'dire_name'])
        radiant_pick, dire_pick = parsePicsBans(match)
        
 
        for hero in radiant_pick:
            match_results[hero] = 1
            match_results_inverse[hero] = -1
        for hero in dire_pick:
            match_results[hero] = -1
            match_results_inverse[hero] = 1
        match_results[0] = match_info['match_id']
        match_results[winner_cell] = match_info['radiant_win']
        match_results[winner_cell - 1] = match_info['dire_name']
        match_results[winner_cell - 2] = match_info['radiant_name']
         
        match_results_inverse[0] = match_info['match_id']
        match_results_inverse[winner_cell] = 1 - match_info['radiant_win']
        match_results_inverse[winner_cell - 1] = match_info['radiant_name']
        match_results_inverse[winner_cell - 2] = match_info['dire_name']

        if match_info['radiant_win'] == None:
            print(match_info['radiant_win'])
    
    except:
        error_count += 1

    only_heroes_data.loc[len(only_heroes_data)] = match_results
    only_heroes_data.loc[len(only_heroes_data)] = match_results_inverse
    
only_heroes_data = only_heroes_data[only_heroes_data.winner != '']
only_heroes_data.dropna(inplace=True)
only_heroes_data = only_heroes_data[only_heroes_data.columns[1:]]


In [ ]:
for index, row in only_heroes_data[only_heroes_data.columns[:-1]].iterrows():
    if sum(row) != 0:
        print sum(row)

In [ ]:
only_heroes_data = only_heroes_data[only_heroes_data.winner != '']
only_heroes_data.dropna(inplace=True)

In [6]:
rank = pd.read_csv('rank_.csv')

In [ ]:
only_heroes_data.to_csv('only_heroes_data.csv')

In [ ]:
def addRanking(data, rank):
    avr_rank = 25
    data['dire_rank'] = avr_rank
    data['radiant_rank'] = avr_rank
    for index, row in data.iterrows():
        if row['dire_name'] in rank['0'].values:
            data.loc[index, 'dire_rank'] = rank[rank['0'] == row['dire_name']]['1'].values[0]
        if row['radiant_name'] in rank['0'].values:
            data.loc[index, 'radiant_rank'] = rank[rank['0'] == row['radiant_name']]['1'].values[0]        
            
    return data

In [ ]:
hdr = addRanking(only_heroes_data, rank)
#only_heroes_data

In [ ]:
hdr = hdr.reindex(columns = np.concatenate([hdr.columns[:-5].values, hdr.columns[-2:].values, hdr.columns[-5:-2].values]))

In [ ]:
del hdr['radiant_name']
del hdr['dire_name']

In [ ]:
hdr.to_csv('hdr.csv')


In [7]:
rates = pd.read_csv('rates_simple.csv')

In [ ]:
hdr.head()

In [ ]:
hdr_close_rates = hdr.loc[abs(hdr['dire_rank'] - hdr['radiant_rank']) < 3,]

In [ ]:
#hdr_close_rates[np.concatenate([hdr_close_rates.columns[:-3].values, hdr_close_rates.columns[-1:]])]

In [ ]:
hdr_close_rates.to_csv('hdr_close_rates.csv')

In [ ]:
del hdr_close_rates['radiant_name']
del hdr_close_rates['dire_name']

In [19]:
hdr_close_rates = pd.read_csv('hdr_close_rates.csv')
del hdr_close_rates['radiant_name']
del hdr_close_rates['dire_name']

# HERE I SHUFFLE DATA

In [8]:
def splitData(data):
    shuffled_data = data.iloc[np.random.permutation(len(data))]
    shuffled_data.reset_index(drop=True)

    train_data = shuffled_data[:int(len(shuffled_data) * 0.7)]
    validation_data = shuffled_data[int(len(shuffled_data) * 0.7):int(len(shuffled_data) * 0.85)]
    test_data = shuffled_data[int(len(shuffled_data) * 0.85):]
    
    return train_data, validation_data, test_data

In [9]:
#only_heroes_train_data, only_heroes_validation_data, only_heroes_test_data = splitData(only_heroes_data)

def prepareAllData(data):
    train_data, validation_data, test_data = splitData(data)

    X_train = train_data[data.columns[:-1]]
    #X_train = X_train.drop(u'Unnamed: 0', axis = 1 )
    Y_train = train_data[data.columns[-1:]].values[:,-1].astype('float')

    X_validation = validation_data[data.columns[:-1]]
    #X_test = X_test.drop(u'Unnamed: 0', axis = 1)
    Y_validation = validation_data[data.columns[-1:]].values[:,-1].astype('float')

    X_test = test_data[data.columns[:-1]]
    #X_test = X_test.drop(u'Unnamed: 0', axis = 1)
    Y_test = test_data[data.columns[-1:]].values[:,-1].astype('float')

    return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

#X_train, Y_train, X_validation, Y_validation, X_test, Y_test = prepareAllData(hdr_close_rates[np.concatenate([hdr_close_rates.columns[:-3].values, hdr_close_rates.columns[-1:]])])


In [20]:
#hdr_crc = cut(hdr_close_rates, 2)
train, train_label, val, val_label, test, test_label = prepareAllData(hdr_close_rates)
#train

In [ ]:
def cut(df, num):
    return df[df.columns[:-num]]

train_cut = cut(train, 2)
test_cut = cut(test, 2)
valid_cut = cut(val, 2)

In [15]:
def countCorrect(pred, label, round_flag = False):
    bin_pred = pred #[round(i) for i in range(len(pred))]
    if round_flag:
        bin_pred = np.zeros(len(pred))
        bin_pred[pred > 0.5] = 1
    count = 0
    for i in bin_pred == label:
        if i:
            count += 1

    return float(count)/len(bin_pred)

In [ ]:
train.head()

# HERE I TRY LOGREG FROM SKLEARN

In [46]:
from sklearn import linear_model
from sklearn import neighbors
from sklearn import svm as svm_classifier

def runLogReg(tr, tr_label, te, te_label):
    logreg = linear_model.LogisticRegression(C=1e5)

    logreg.fit(tr.values, tr_label)


    logreg_pred = logreg.predict_proba(te)[:, 1]

    print('logreg: {0}'.format(countCorrect(logreg_pred, te_label, True)))
    
    return logreg_pred, logreg

def runSVM(tr, tr_label, te, te_label):
    svm = svm_classifier.SVC(kernel='rbf', probability=True,)
    svm.fit(tr, tr_label)


    svm_pred = svm.predict_proba(te)[:, 1]
    print(svm_pred[:10])
    print('svm: {0}'.format(countCorrect(svm_pred, te_label, True)))
    
    return svm_pred, svm
    
def runNN(tr, tr_label, te, te_label, n_neighbors = 5):
    #n_neighbors = 3
    nn = neighbors.KNeighborsClassifier(n_neighbors)
    nn.fit(tr, tr_label)

    nn_pred = nn.predict_proba(te)[:, 1]

    print('NN: {0}'.format(countCorrect(nn_pred, te_label, True)))
    
    return nn_pred, nn

In [58]:
def runFullStack(train, train_label, test, test_label):
    #nn_pred, nn = runNN(train, train_label, test, test_label)
    svm_pred, svm = runSVM(train, train_label, test, test_label)
    logreg_pred, logreg = runLogReg(train, train_label, test, test_label)
    return svm_pred, svm, logreg_pred, logreg

In [48]:
nn_pred, nn, svm_pred, svm, logreg_pred, logreg = runFullStack(train, train_label, test, test_label)


NameError: name 'train' is not defined

In [ ]:
t = train[7:8].values
logreg.predict_proba(t)

In [ ]:
t[t == 1.1] = 2

In [ ]:
t[t == 1] = 1

In [ ]:
svm.predict_proba(t)

In [ ]:
t

# HERE WE GO WITH XG BOOST

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(train_cut, train_label)
dvalidation = xgb.DMatrix(valid_cut, val_label)
dtest = xgb.DMatrix(test_cut, test_label)

In [ ]:
# specify parameters via map, definition are same as c++ version
param = {'max_depth':5, 'eval_metric' : 'logloss', 'eta':0.1, }

# specify validations set to watch performance
watchlist  = [(dvalidation,'eval'), (dtrain,'train')]
num_round = 6
bst = xgb.train(param, dtrain, num_round, watchlist, verbose_eval=True)

# this is prediction
#preds = bst.predict(dtest)
#labels = dtest.get_label()

In [ ]:
bst_preds = bst.predict(dtest)

In [ ]:
print(countCorrect(bst_preds, test_label, True))

In [ ]:
bin_pred_asasa = np.zeros(len(bst_preds))
bin_pred_asasa[bst_preds > 0.5] = 1

In [ ]:
#bin_pred_asasa == test_label

# PYBRAIN MSFK


In [ ]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer

In [ ]:
net = buildNetwork(115, 5, 1)
ds = SupervisedDataSet(115, 1)

In [ ]:
test.values.shape

In [ ]:
#train = X_train
train['winner'] = train_label
val['winner'] = val_label
#test = X_test
test['winner'] = test_label

In [ ]:
for row in train.values:
    #print(row[:-1])
    indata = row[:-1]
    outdata = row[len(row) - 1]
    ds.addSample(indata,outdata)

In [ ]:
trainer = BackpropTrainer(net, ds)
#trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01)
trainer.trainEpochs(7)

In [ ]:
NNpred = []
for row in test.values:
    NNpred.append(net.activate(row[:-1]))


In [ ]:
binaryNNpred = [round(i) for i in NNpred]


In [ ]:
count = 0
binaryNNpred == test_label:
    if i:
        count += 1

print(float(count)/len(binaryNNpred))
print(count)

In [ ]:
def calcErr(pred, label):
    binaryNNpred = [round(i) for i in pred]
    count = 0
    for i in range(len(label)):
        if binaryNNpred[i] == label[i]:
            count += 1

    return (float(count)/len(binaryNNpred))


def runNN(train_data, valid_data, test_data, rounds):
    net = buildNetwork(len(train_data.columns[:-1]), 10, 1)
    ds = SupervisedDataSet(len(train_data.columns[:-1]), 1)
    
    for row in train_data.values:
        indata = row[:-1]
        outdata = row[len(row) - 1]
        ds.addSample(indata,outdata)
    trainer = BackpropTrainer(net, ds, learningrate=0.05)
    #trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01)
    
    NNpred = []
    for row in valid_data.values:
        NNpred.append(net.activate(row[:-1]))
    label = valid_data.winner.values.astype('float')
    
    err_curr = calcErr(NNpred, label)
    err_prev = 1000
    #while err_curr >= err_prev:
    for i in range(rounds):
        trainer.trainEpochs(1)
    
        NNpred = []
        for row in valid_data.values:
            NNpred.append(net.activate(row[:-1]))
        
        label = valid_data.winner.values.astype('float')
    
        err_prev = err_curr
        err_curr = calcErr(NNpred, label)
        print ('curr: {0}'.format(err_curr))
        print ('prev: {0}'.format(err_prev))

    return err_curr


In [ ]:
#print runNN(train, val, test, 15)

In [ ]:
#json.loads(alldata[9])

In [ ]:
def searchFor31(picksbans, hero_id):
    for hero in picksbans:
        if hero['hero_id'] == hero_id:
            return True
    return False

def checkHero(hero_id):
    for match_string in alldata:
        match = json.loads(match_string)
        try:
            if searchFor31(match['picks_bans'], hero_id):
                return True
        except:
            pass
    return False

for i in range(130):
    if not checkHero(i):
        print i

checkHero(200)

In [ ]:
json.loads(alldata[0])

In [ ]:
winrates = pd.read_csv('TimeWinRates.csv')

# FACTORIZATION MACHINE

In [44]:
from kaggler.online_model import FM
import scipy

def runFM(tr, tr_label, te, te_label):
    fm = FM(n=len(tr.columns), # number of features
    epoch=20, # number of epochs
    dim=4, # size of factors for interactions
    a=.1) 
    
    fm.fit(scipy.sparse.csr_matrix(tr.values), tr_label)

    fm_pred = fm.predict(scipy.sparse.csr_matrix(te.values))
    print(countCorrect(fm_pred, te_label, True))
    return fm_pred

In [ ]:
hdr.head()

# REFACTOR ONE VERY HOT


In [ ]:
double_hdr = pd.DataFrame(columns = ['r_hero_'+str(i) for i in range(114)] + ['d_hero_'+str(i) for i in range(114)] + ['dire_rank', 'radiant_rank', 'winner'])


In [ ]:
double_hdr = pd.DataFrame(columns = ['r_hero_'+str(i) for i in range(114)] + ['d_hero_'+str(i) for i in range(114)] + ['dire_rank', 'radiant_rank', 'winner'], index = hdr.index)
c = 0
for index, row in hdr.iterrows():
    radiant_pick = []
    dire_pick = []

    for i in range(len(row.values[:-1])):
        if row.values[i] == 1:
            radiant_pick.append(i + 1)
        elif row.values[i] == -1:
            dire_pick.append(i + 1)

    double_hdr.loc[len(double_hdr)] = np.zeros(len(double_hdr.columns))
    for i in dire_pick: 
        #pass
        double_hdr.loc[index, 'd_hero_'+str(i)] = 1
    for i in radiant_pick: 
    #    #pass
        double_hdr.loc[index, 'r_hero_'+str(i)] = 1
    
    double_hdr.loc[index, 'radiant_rank'] = row['radiant_rank']
    double_hdr.loc[index, 'dire_rank'] = row['dire_rank']
    if int(row['winner']) == 1:
        double_hdr.loc[index, 'winner'] = 1
    else:
        double_hdr.loc[index, 'winner'] = 0
    #double_hdr.loc[index, 'winner'] = row['winner']

In [ ]:
double_hdr.to_csv('double_hdr.csv')

In [239]:
double_hdr = pd.read_csv('double_hdr.csv')

In [240]:
#for index, row in hdr.iterrows():

#    if row['winner'] == 1:
#        print(row['winner'])
#sum(double_hdr[:500].winner.values)
double_hdr = double_hdr.fillna(0)

In [241]:
del double_hdr['dire_rank']
del double_hdr['radiant_rank']

In [242]:
del double_hdr['Unnamed: 0']

In [243]:
d_train, d_train_label, d_valid, d_valid_label, d_test, d_test_label = prepareAllData(double_hdr)


In [244]:
svm_pred, svm, logreg_pred, logreg  =  runFullStack(d_train, d_train_label, d_test, d_test_label)

[ 0.51060022  0.43870837  0.51559046  0.48228544  0.42309435  0.46430858
  0.46517726  0.54476715  0.46540746  0.53015432]
svm: 0.57269700333
logreg: 0.583795782464


In [ ]:
#runFullStack(d_train, d_train_label, d_test, d_test_label)

runFM(d_train, d_train_label, d_test, d_test_label)

In [90]:
poly_and_rates = pd.read_csv('TimeWinRatesPoly.csv')


Index([u'Unnamed: 0', u'hero_id', u'General Rate', u'Early', u'Mid', u'Late',
       u'Poly1', u'Poly2', u'Poly3', u'Poly4'],
      dtype='object')

In [229]:
polies = [[] for i in range(115)]

for index, line in poly_and_rates.iterrows():
    
    pol = [line[6], line[7], line[8], line[9]]
    polies[int(line[1])] = np.poly1d(pol)



In [157]:
t = d_test[8:9].values[0]
print(logreg.predict_proba(t.reshape(1, -1)))
for i in range(114, 228):
    if t[i] == 1:
        t[i] = 2

print(logreg.predict_proba(t.reshape(1, -1)))

In [247]:
from sklearn.externals import joblib
joblib.dump(svm, './ready_models/svm_picks_rates.pkl') 
joblib.dump(logreg, './ready_models/logreg_picks_rates.pkl') 

['./ready_models/logreg_picks_rates.pkl',
 './ready_models/logreg_picks_rates.pkl_01.npy',
 './ready_models/logreg_picks_rates.pkl_02.npy',
 './ready_models/logreg_picks_rates.pkl_03.npy',
 './ready_models/logreg_picks_rates.pkl_04.npy']

In [172]:
def calcTeamWinRate(time, pick, polies):
    rates = [polies[i](time) for i in pick]
    return sum(rates)

In [ ]:
def addRanking(data, rank):
    avr_rank = 25
    data['dire_rank'] = avr_rank
    data['radiant_rank'] = avr_rank
    for index, row in data.iterrows():
        if row['dire_name'] in rank['0'].values:
            data.loc[index, 'dire_rank'] = rank[rank['0'] == row['dire_name']]['1'].values[0]
        if row['radiant_name'] in rank['0'].values:
            data.loc[index, 'radiant_rank'] = rank[rank['0'] == row['radiant_name']]['1'].values[0]        
            
    return data

In [218]:
rank = pd.read_csv('rank_.csv')


In [234]:

total_count = 0
cool_count = 0
for match_string in alldata:
    match = json.loads(match_string)
    
    if True:
        match_info =  parseMatch(match, ['match_id', 'radiant_win', 'radiant_name', 'dire_name', 'duration'])
        radiant_pick, dire_pick = parsePicsBans(match)
        
        if match_info['radiant_win']:
            known_winner = 1
        else: 
            known_winner = 0
            
        dire_rank = 25
        radiant_rank = 25
        if match_info['radiant_name'] in rank['0'].values:
            radiant_rank = rank[rank['0'] == match_info['radiant_name'] ]['1'].values[0]
        if match_info['dire_name'] in rank['0'].values:
            dire_rank = rank[rank['0'] == match_info['dire_name'] ]['1'].values[0]

        if abs(dire_rank - radiant_rank) < 3: 
            total_count += 1
            radiant_prop = calcTeamWinRate(match_info['duration'], radiant_pick, polies)
            dire_prop = calcTeamWinRate(match_info['duration'], dire_pick, polies)
            
            if radiant_prop > dire_prop:
                winner = 1
            else:
                winner = 0
                
            if winner == known_winner:
                cool_count+=1
                
    else:
        error_count += 1

'CDEC Gaming' in rank['0'].values

/home/anton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/anton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


True

In [235]:
float(cool_count)/total_count

0.616

In [206]:
json.loads(alldata[0])['duration']

2446

In [227]:
len(polies)

111

In [238]:
double_hdr.r_hero_24.value_counts()

0    6002
Name: r_hero_24, dtype: int64